In [1]:
# read data from data set
from utils.data_utils import jigsaw_toxix_ds_get_df
import numpy as np
from config import *
df = jigsaw_toxix_ds_get_df()
comments = df["comment_text"].tolist()
comments = [x.replace('\n', ' ') for x in comments]
dump_text = '\n'.join(comments)
total_classes = 6
class_matrix = df[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]].astype('int')
label_matrix = class_matrix.values

print(len(label_matrix))

159571


In [2]:
# load word2vec model
import os 
from config import model_folder
from model_factory.embeddings.w2v import w2v_load_from_keyedvectors, build_embedding_layer
w2v_name = 'google_keyed_vector_format.bin'
model_path = os.path.join(model_folder, w2v_name)
    
w2v_model = w2v_load_from_keyedvectors(model_path)
vocab = w2v_model.vocab

word_2_idx = dict(zip(vocab.keys(), range(len(vocab))))
max_sent_length = 80
trainable = False
embedding_layer = build_embedding_layer(w2v_model, word_2_idx, len(vocab), max_sent_length, trainable)
print(embedding_layer)

Using TensorFlow backend.


In [3]:
# tokenization
import nltk
from keras.preprocessing.sequence import pad_sequences



def tokenizer(text, word2idx, max_len=80, total=None):
    if total is not None:
        text = text[:total]
    for sentence_idx in range(len(text)):
        sentence = text[sentence_idx]
        text[sentence_idx] = nltk.word_tokenize(sentence)
 
    def _sent_to_idx(s, w2i):
        for word_idx in range(len(s)):
            word = s[word_idx]
            idx = w2i.get(word, w2i['null'])
            s[word_idx] = idx
        return s
        
    for sentence_idx in range(len(text)):
        sentence = text[sentence_idx]
        sequence = _sent_to_idx(sentence, word2idx)
        text[sentence_idx] = sequence
    
    text = pad_sequences(text, maxlen=max_len, value=word2idx['null'])   
    return text

num_samples = None
tokenized_sequence = tokenizer(comments, word_2_idx, max_sent_length, total=num_samples)
if num_samples is not None:
    label_matrix = label_matrix[:num_samples]


In [4]:
# build model
from keras import models, layers, objectives, metrics
h_dim = 300
m = models.Sequential()
m.add(embedding_layer)
m.add(layers.LSTM(
    units=h_dim, 
    return_sequences=True,
))
m.add(layers.LSTM(
    units=h_dim, 
    return_sequences=False,
))
m.add(layers.Dense(total_classes, activation='sigmoid'))
m.compile(optimizer='adam', loss=objectives.binary_crossentropy, metrics=[metrics.top_k_categorical_accuracy])
print(len(tokenized_sequence), len(label_matrix))


159571 159571


In [5]:


# join embeddings with data set dataframe
train_test_split = 0.2
m.fit(tokenized_sequence, label_matrix, epochs=20, batch_size=1024, validation_split=train_test_split)
divider = int(len(tokenized_sequence)*train_test_split)

Train on 127656 samples, validate on 31915 samples
Epoch 1/20


  1024/127656 [..............................] - ETA: 3:43 - loss: 0.6907 - top_k_categorical_accuracy: 0.9775

  2048/127656 [..............................] - ETA: 2:07 - loss: 0.6553 - top_k_categorical_accuracy: 0.9795

  3072/127656 [..............................] - ETA: 1:35 - loss: 0.6082 - top_k_categorical_accuracy: 0.9863

  4096/127656 [..............................] - ETA: 1:19 - loss: 0.5346 - top_k_categorical_accuracy: 0.9897

...

123904/127656 [============================>.] - ETA: 1s - loss: 0.0557 - top_k_categorical_accuracy: 1.0000

124928/127656 [============================>.] - ETA: 0s - loss: 0.0557 - top_k_categorical_accuracy: 1.0000

125952/127656 [============================>.] - ETA: 0s - loss: 0.0557 - top_k_categorical_accuracy: 1.0000

126976/127656 [============================>.] - ETA: 0s - loss: 0.0558 - top_k_categorical_accuracy: 1.0000

127656/127656 [==============================] - 38s 298us/step - loss: 0.0558 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0709 - val_top_k_categorical_accuracy: 0.9999


In [30]:
divider = int(len(tokenized_sequence)*train_test_split)

test_X = tokenized_sequence[-divider:]
test_Y = label_matrix[-divider:]
pred_Y = m.predict(test_X)
topic_Y = test_Y.T
pred_topic_Y = pred_Y.T


In [47]:
# evaluate performance
threshold = 0.5
print(threshold)
from sklearn.metrics.classification import precision_score, recall_score
print(','.join(['sample size','precision','recall', 'prior']))
# print(pred_topic_Y[0].tolist())
for topic_idx in range(len(pred_topic_Y)):
    true_topic = topic_Y[topic_idx]
    pred_topic = pred_topic_Y[topic_idx]
    pred_topic[np.where(pred_topic >= threshold)] = 1
    pred_topic[np.where(pred_topic < threshold)] = 0
    print('%d, %.2f, %.2f, %.2f' % (sum(true_topic), precision_score(true_topic, pred_topic), recall_score(true_topic, pred_topic), sum(true_topic)/len(true_topic)))


0.5
sample size,precision,recall,prior
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

/home/pintellect/anaconda3/envs/ml-dev/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
# fit model and test